# Реализуем классическую стратегию трейдинга основанную на торговле пересечений длинной и короткой MA.
В результате стратегия вычисляет долгосрочный тренд актива, будь он восходящим или нисходящим.

In [229]:
! pip install vectorbt
! pip install yfinance
import vectorbt as vbt

import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

In [230]:
#Для наблюдений возьмем BTC, тк он является повадырем для всех остальных монет и между тем
#самой стабильной криптовалютой, если не брать в счет стэйблкоины
crypto = vbt.YFData.download(
    'BTC-USD',
    interval='1d',
    missing_index='drop',
    start='2016-01-01',
    end = '2021-01-01'
).get('Close')

In [231]:
fast_ma = vbt.MA.run(crypto, 50, short_name='fast MA')
slow_ma = vbt.MA.run(crypto, 200, short_name='fast MA')

In [232]:
#входим в лонг, когда MA200 пересекает MA50 сверху
#в шорт входим при обратной ситуации
long_entries = fast_ma.ma_crossed_above(slow_ma)
short_entries = fast_ma.ma_crossed_below(slow_ma)

In [233]:
pf = vbt.Portfolio.from_signals(
    crypto,
    entries = long_entries,
    short_entries = short_entries, 
    init_cash = 20000000,
    fees = 0.001,
    freq = '1d'
)

In [234]:
pf.stats()

Start                         2015-12-31 00:00:00+00:00
End                           2020-12-31 00:00:00+00:00
Period                               1828 days 00:00:00
Start Value                                  20000000.0
End Value                               54851474.084646
Total Return [%]                              174.25737
Benchmark Return [%]                        6635.704696
Max Gross Exposure [%]                            100.0
Total Fees Paid                           330055.893309
Max Drawdown [%]                              76.056448
Max Drawdown Duration                 554 days 00:00:00
Total Trades                                          6
Total Closed Trades                                   5
Total Open Trades                                     1
Open Trade PnL                          37657798.899616
Win Rate [%]                                       40.0
Best Trade [%]                                68.901884
Worst Trade [%]                              -36

In [235]:
print(f'The Beta of the strategy is {pf.beta()*100:.3f}%')

The Beta of the strategy is 7.466%


In [236]:
fig = pf.plot(subplots=['orders','trade_pnl','cum_returns'])
fast_ma.ma.vbt.plot(fig=fig)
slow_ma.ma.vbt.plot(fig=fig)
fig.show()

# Попробуем перебрать параметры кривых и посмотреть на эффективность стратегии

In [237]:
short_MA = 0
long_MA = 0
beta = 0.3
max_drawdown = -1

short_drawdown = 0
long_drawdown = 0

for short in range(10, 110, 10):
    for long in range(short + 10, 210, 10):
        fast_ma = vbt.MA.run(crypto, short, short_name='fast MA')
        slow_ma = vbt.MA.run(crypto, long, short_name='fast MA')

        long_entries = fast_ma.ma_crossed_above(slow_ma)
        short_entries = fast_ma.ma_crossed_below(slow_ma)

        pf = vbt.Portfolio.from_signals(
            crypto,
            entries = long_entries,
            short_entries = short_entries, 
            init_cash = 20000000,
            fees = 0.001,
            freq = '1d'
        )
        
        if (pf.beta() < beta
            and pf.total_return() > 0): #Оптимизация параметров по Beta
            short_MA = short
            long_MA = long
            beta = pf.beta()
            total_return = pf.total_return()
            
        if (pf.max_drawdown() > max_drawdown): #Оптимизация параметров по Max drawdown
            max_drawdown = pf.max_drawdown()
            short_drawdown = short
            long_drawdown = long

print("Short_MA:", short_MA)
print("Long_MA:", long_MA)
print()
print("Short_drawdown:", short_drawdown)
print("Long_drawdown:", long_drawdown)

Short_MA: 90
Long_MA: 200

Short_drawdown: 100
Long_drawdown: 190


In [238]:
fast_ma = vbt.MA.run(crypto, short_MA, short_name='fast MA') #short_MA = 90
slow_ma = vbt.MA.run(crypto, long_MA, short_name='fast MA') #long_MA = 200

long_entries = fast_ma.ma_crossed_above(slow_ma)
short_entries = fast_ma.ma_crossed_below(slow_ma)

pf = vbt.Portfolio.from_signals(
    crypto,
    entries = long_entries,
    short_entries = short_entries, 
    init_cash = 20000000,
    fees = 0.001,
    freq = '1d'
)
pf.stats()

Start                         2015-12-31 00:00:00+00:00
End                           2020-12-31 00:00:00+00:00
Period                               1828 days 00:00:00
Start Value                                  20000000.0
End Value                              104369661.184984
Total Return [%]                             421.848306
Benchmark Return [%]                        6635.704696
Max Gross Exposure [%]                            100.0
Total Fees Paid                           402420.979694
Max Drawdown [%]                              44.291216
Max Drawdown Duration                 284 days 00:00:00
Total Trades                                          6
Total Closed Trades                                   5
Total Open Trades                                     1
Open Trade PnL                          70010463.430938
Win Rate [%]                                       80.0
Best Trade [%]                                50.846946
Worst Trade [%]                              -32

In [239]:
print(f'The Beta of the strategy is {pf.beta()*100:.3f}%')

The Beta of the strategy is -5.921%


In [240]:
#Посмотрим на результаты перебора и проверим стратегию в промежутке с 2021 года до сегодняшнего дня
crypto = vbt.YFData.download(
    'BTC-USD',
    interval='1d',
    missing_index='drop',
    start='2021-01-01',
    end = dt.datetime.today()
).get('Close')

fast_ma = vbt.MA.run(crypto, short_MA, short_name='fast MA') #short_MA = 90
slow_ma = vbt.MA.run(crypto, long_MA, short_name='fast MA') #long_MA = 200

long_entries = fast_ma.ma_crossed_above(slow_ma)
short_entries = fast_ma.ma_crossed_below(slow_ma)

pf = vbt.Portfolio.from_signals(
    crypto,
    entries = long_entries,
    short_entries = short_entries, 
    init_cash = 20000000,
    fees = 0.001,
    freq = '1d'
)

In [241]:
pf.stats()

Start                         2020-12-31 00:00:00+00:00
End                           2023-04-22 00:00:00+00:00
Period                                843 days 00:00:00
Start Value                                  20000000.0
End Value                               24482421.048628
Total Return [%]                              22.412105
Benchmark Return [%]                          -5.735321
Max Gross Exposure [%]                            100.0
Total Fees Paid                            77726.734979
Max Drawdown [%]                              48.154663
Max Drawdown Duration                 493 days 00:00:00
Total Trades                                          3
Total Closed Trades                                   2
Total Open Trades                                     1
Open Trade PnL                           3686957.650191
Win Rate [%]                                       50.0
Best Trade [%]                                 43.94618
Worst Trade [%]                               -2

In [242]:
print(f'The Beta of the strategy is {pf.beta()*100:.3f}%')

The Beta of the strategy is -4.021%


In [243]:
fig = pf.plot(subplots=['orders','trade_pnl','cum_returns'])
fast_ma.ma.vbt.plot(fig=fig)
slow_ma.ma.vbt.plot(fig=fig)
fig.show()

Из результатов перебора параметров стратегии видно, что получилось хорошо оптимизировать на трейне Beta и при этом оставить Total return > 0. В итоге результаты на тесте получились достаточно неплохими.
(если вместе или по отдельнсти от Beta оптимизировать еще и Sharpe ratio или Total return, то в результате перебора получатся параметры, при которых стратегия сливает депо на трейне, поэтому я остановился на оптимизации лишь Beta)
Но при такой оптимизации пострадали Max Drawdown и Max Drawdown Duration.

При переборе я также подобрал такие параметры short_MA и long_MA, при которых Max Drawdown и Max Drawdown Duration на трейне были наилучшими.
Посмотрим на результаты торговли такой системы с параметрами short_MA = 100 и long_MA = 190

In [244]:
fast_ma = vbt.MA.run(crypto, short_drawdown, short_name='fast MA') #short_MA = 10
slow_ma = vbt.MA.run(crypto, long_drawdown, short_name='fast MA') #long_MA = 190

long_entries = fast_ma.ma_crossed_above(slow_ma)
short_entries = fast_ma.ma_crossed_below(slow_ma)

pf = vbt.Portfolio.from_signals(
    crypto,
    entries = long_entries,
    short_entries = short_entries, 
    init_cash = 20000000,
    fees = 0.001,
    freq = '1d'
)

In [245]:
pf.stats()

Start                         2020-12-31 00:00:00+00:00
End                           2023-04-22 00:00:00+00:00
Period                                843 days 00:00:00
Start Value                                  20000000.0
End Value                                4424163.492096
Total Return [%]                             -77.879183
Benchmark Return [%]                          -5.735321
Max Gross Exposure [%]                            100.0
Total Fees Paid                            70319.165049
Max Drawdown [%]                              90.603116
Max Drawdown Duration                 641 days 00:00:00
Total Trades                                          4
Total Closed Trades                                   3
Total Open Trades                                     1
Open Trade PnL                            613656.040026
Win Rate [%]                                  33.333333
Best Trade [%]                                46.190509
Worst Trade [%]                              -81

In [246]:
print(f'The Beta of the strategy is {pf.beta()*100:.3f}%')

The Beta of the strategy is -29.612%


In [247]:
fig = pf.plot(subplots=['orders','trade_pnl','cum_returns'])
fast_ma.ma.vbt.plot(fig=fig)
slow_ma.ma.vbt.plot(fig=fig)
fig.show()

Из результатов прогона стратегии с параметрами short_MA = 100 и long_MA = 190 на тесте, мы видим, что стратегия не то что не зарабатывает - она также сливает депозит(-77% за 2 года) и при этом всем еще и имеет параметры Max Drawdown и Max Drawdown Duration хуже, чем для случая, когда эти 2 параметра не оптимизировались от слова совсем. (90% и 641 день против 48% и 493 дней)

Из результатов тестирования можно сделать вывод, что для данной стратегии не получится оптимизировать на трейне Max Drawdown, Max Drawdown Duration, Sharpe ratio и Total return, чтобы Total return на тесте был > 0.

Вполне возможно лично я не умею оптимизировать параметры так, чтобы значения были в среднем все достаточно хороши, поэтому с нетерпением жду от Вас фидбека.

p.s. Хоть здесь это и не представлено, но я отдельно пытался подогнать параметры стратегии так, чтобы именно Max Drawdown был меньше 30%, но таких пар параметров не нашлось. Ровно также я пытался увеличить и Total return и Sharpe ratio, но при подобранных параметрах стратегия сливала депозит на тесте.

# Результаты
Мы получили 2 кривые: 90MA и 200MA

Результаты на тесте(период с 01-01-2021 по 22.04-2023):  
Total return 22.4%   
Beta = -4.021%  
Sharpe Ratio = 0.409547  
Max Drawdown = 45%  
Max Drawdown = 526 дней